In [1]:
import pandas as pd

In [2]:
tennis_df = pd.DataFrame()
for year in range(2001, 2022):
    
    file = ""
    if year < 2013:
        file = f"{year}.xls"
        print(file)
    else:
        assert(year >= 2013)
        file = f"{year}.xlsx"
        print(file)
    
    tennis_df = tennis_df.append(pd.read_excel(file))

2001.xls
2002.xls
2003.xls
2004.xls
2005.xls
2006.xls
2007.xls
2008.xls
2009.xls
2010.xls
2011.xls
2012.xls
2013.xlsx


C:\Users\briel\anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


2014.xlsx
2015.xlsx
2016.xlsx
2017.xlsx
2018.xlsx
2019.xlsx
2020.xlsx
2021.xlsx


In [3]:
pd.set_option('display.max_columns', None)

In [4]:
tennis_df.head(30)

,ATP,Location,Tournament,Date,Series,Court,Surface,Round,Best of,Winner,Loser,WRank,LRank,W1,L1,W2,L2,W3,L3,W4,L4,W5,L5,Wsets,Lsets,Comment,CBW,CBL,GBW,GBL,IWW,IWL,SBW,SBL,B365W,B365L,B&WW,B&WL,EXW,EXL,PSW,PSL,WPts,LPts,UBW,UBL,LBW,LBL,SJW,SJL,MaxW,MaxL,AvgW,AvgL
0,1,Adelaide,AAPT Championships,2001-01-01,International,Outdoor,Hard,1st Round,3,Clement A.,Gaudenzi A.,18.0,101,6.0,7.0,6.0,0.0,6.0,3.0,NaN,NaN,NaN,NaN,2.0,1.0,Completed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Adelaide,AAPT Championships,2001-01-01,International,Outdoor,Hard,1st Round,3,Goldstein P.,Jones A.,81.0,442,6.0,4.0,6.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,Completed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,Adelaide,AAPT Championships,2001-01-01,International,Outdoor,Hard,1st Round,3,Haas T.,Smith L.,23.0,485,6.0,3.0,6.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,Completed,NaN,NaN,1.12,4.00,1.10,3.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,Adelaide,AAPT Championships,2001-01-01,International,Outdoor,Hard,1st Round,3,Henman T.,Rusedski G.,10.0,69,6.0,2.0,7.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,Completed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,Adelaide,AAPT Championships,2001-01-01,International,Outdoor,Hard,1st Round,3,Hewitt L.,Arthurs W.,7.0,83,6.0,4.0,6.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,Completed,1.28,3.30,1.12,3.80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1,Adelaide,AAPT Championships,2001-01-01,International,Outdoor,Hard,1st Round,3,Johansson T.,Mantilla F.,39.0,99,7.0,6.0,6.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,Completed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1,Adelaide,AAPT Championships,2001-01-01,International,Outdoor,Hard,1st Round,3,Ljubicic I.,Sargsian S.,91.0,94,6.0,2.0,7.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,Completed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1,Adelaide,AAPT Championships,2001-01-01,International,Outdoor,Hard,1st Round,3,Malisse X.,Woodruff C.,127.0,67,2.0,6.0,6.0,3.0,6.0,2.0,NaN,NaN,NaN,NaN,2.0,1.0,Completed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1,Adelaide,AAPT Championships,2001-01-01,International,Outdoor,Hard,1st Round,3,Martin A.,Puentes G.,79.0,92,6.0,2.0,6.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,Completed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1,Adelaide,AAPT Championships,2001-01-01,International,Outdoor,Hard,1st Round,3,Massu N.,Sluiter R.,87.0,95,6.0,2.0,6.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,Completed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
tennis_df.tail()

,ATP,Location,Tournament,Date,Series,Court,Surface,Round,Best of,Winner,Loser,WRank,LRank,W1,L1,W2,L2,W3,L3,W4,L4,W5,L5,Wsets,Lsets,Comment,CBW,CBL,GBW,GBL,IWW,IWL,SBW,SBL,B365W,B365L,B&WW,B&WL,EXW,EXL,PSW,PSL,WPts,LPts,UBW,UBL,LBW,LBL,SJW,SJL,MaxW,MaxL,AvgW,AvgL
2387,58,Vienna,Vienna Open,2021-10-29,ATP500,Indoor,Hard,Quarterfinals,3,Tiafoe F.,Schwartzman D.,49.0,16.0,6.0,4.0,7.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,Completed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.20,1.66,NaN,NaN,NaN,NaN,2.20,1.75,1192.0,2693.0,NaN,NaN,NaN,NaN,NaN,NaN,2.35,1.75,2.21,1.67
2388,58,Vienna,Vienna Open,2021-10-29,ATP500,Indoor,Hard,Quarterfinals,3,Sinner J.,Ruud C.,11.0,8.0,7.0,5.0,6.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,Completed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.40,3.00,NaN,NaN,NaN,NaN,1.38,3.31,3260.0,3615.0,NaN,NaN,NaN,NaN,NaN,NaN,1.45,3.35,1.39,3.00
2389,58,Vienna,Vienna Open,2021-10-30,ATP500,Indoor,Hard,Semifinals,3,Zverev A.,Alcaraz C.,4.0,42.0,6.0,3.0,6.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,Completed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.28,3.75,NaN,NaN,NaN,NaN,1.27,4.21,6680.0,1355.0,NaN,NaN,NaN,NaN,NaN,NaN,1.39,4.21,1.30,3.56
2390,58,Vienna,Vienna Open,2021-10-30,ATP500,Indoor,Hard,Semifinals,3,Tiafoe F.,Sinner J.,49.0,11.0,3.0,6.0,7.0,5.0,6.0,2.0,NaN,NaN,NaN,NaN,2.0,1.0,Completed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.00,1.25,NaN,NaN,NaN,NaN,4.18,1.27,1192.0,3260.0,NaN,NaN,NaN,NaN,NaN,NaN,4.33,1.30,3.86,1.26
2391,58,Vienna,Vienna Open,2021-10-31,ATP500,Indoor,Hard,The Final,3,Zverev A.,Tiafoe F.,4.0,49.0,7.0,5.0,6.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,Completed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.20,4.50,NaN,NaN,NaN,NaN,1.21,5.12,6680.0,1192.0,NaN,NaN,NaN,NaN,NaN,NaN,1.24,5.50,1.19,4.74


In [6]:
tennis_df

,ATP,Location,Tournament,Date,Series,Court,Surface,Round,Best of,Winner,Loser,WRank,LRank,W1,L1,W2,L2,W3,L3,W4,L4,W5,L5,Wsets,Lsets,Comment,CBW,CBL,GBW,GBL,IWW,IWL,SBW,SBL,B365W,B365L,B&WW,B&WL,EXW,EXL,PSW,PSL,WPts,LPts,UBW,UBL,LBW,LBL,SJW,SJL,MaxW,MaxL,AvgW,AvgL
0,1,Adelaide,AAPT Championships,2001-01-01,International,Outdoor,Hard,1st Round,3,Clement A.,Gaudenzi A.,18.0,101,6.0,7.0,6.0,0.0,6.0,3.0,NaN,NaN,NaN,NaN,2.0,1.0,Completed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Adelaide,AAPT Championships,2001-01-01,International,Outdoor,Hard,1st Round,3,Goldstein P.,Jones A.,81.0,442,6.0,4.0,6.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,Completed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,Adelaide,AAPT Championships,2001-01-01,International,Outdoor,Hard,1st Round,3,Haas T.,Smith L.,23.0,485,6.0,3.0,6.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,Completed,NaN,NaN,1.12,4.0,1.1,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,Adelaide,AAPT Championships,2001-01-01,International,Outdoor,Hard,1st Round,3,Henman T.,Rusedski G.,10.0,69,6.0,2.0,7.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,Completed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,Adelaide,AAPT Championships,2001-01-01,International,Outdoor,Hard,1st Round,3,Hewitt L.,Arthurs W.,7.0,83,6.0,4.0,6.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,Completed,1.28,3.3,1.12,3.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2387,58,Vienna,Vienna Open,2021-10-29,ATP500,Indoor,Hard,Quarterfinals,3,Tiafoe F.,Schwartzman D.,49.0,16.0,6.0,4.0,7.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,Completed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.20,1.66,NaN,NaN,NaN,NaN,2.20,1.75,1192.0,2693.0,NaN,NaN,NaN,NaN,NaN,NaN,2.35,1.75,2.21,1.67
2388,58,Vienna,Vienna Open,2021-10-29,ATP500,Indoor,Hard,Quarterfinals,3,Sinner J.,Ruud C.,11.0,8.0,7.0,5.0,6.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,Completed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.40,3.00,NaN,NaN,NaN,NaN,1.38,3.31,3260.0,3615.0,NaN,NaN,NaN,NaN,NaN,NaN,1.45,3.35,1.39,3.00
2389,58,Vienna,Vienna Open,2021-10-30,ATP500,Indoor,Hard,Semifinals,3,Zverev A.,Alcaraz C.,4.0,42.0,6.0,3.0,6.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,Completed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.28,3.75,NaN,NaN,NaN,NaN,1.27,4.21,6680.0,1355.0,NaN,NaN,NaN,NaN,NaN,NaN,1.39,4.21,1.30,3.56
2390,58,Vienna,Vienna Open,2021-10-30,ATP500,Indoor,Hard,Semifinals,3,Tiafoe F.,Sinner J.,49.0,11.0,3.0,6.0,7.0,5.0,6.0,2.0,NaN,NaN,NaN,NaN,2.0,1.0,Completed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.00,1.25,NaN,NaN,NaN,NaN,4.18,1.27,1192.0,3260.0,NaN,NaN,NaN,NaN,NaN,NaN,4.33,1.30,3.86,1.26


In [7]:
# sheets = pd.read_excel(file, sheet_name=None, header=None)
#         sheet_list = list(sheets.keys())

#         for i in range(len(sheet_list)):
#             sheet = sheets[sheet_list[i]] #gets a single sheet
#             if i == 0:
#                 headers = sheet.iloc[1, :]
#                 sheet.columns = headers
#                 sheet = sheet.drop(sheet.head(2).index)
#             if i == len(sheet_list) - 1: 
#                 sheet = sheet.drop(sheet.tail(1).index)
#             else:
#                 sheet = sheet.drop(sheet.head(1).index)
#             sheet.columns = headers
#             dataframes.append(sheet)

#     df = pd.concat(dataframes)
#     df = df.rename(columns = lambda x : x.strip())

In [8]:
tennis_df.to_excel("tennis_data_2001_2021.xlsx") 